<a href="https://colab.research.google.com/github/olafthiele/oop22/blob/main/OOP_22_H%C3%B6ren_2_wav2vec_gro%C3%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hören 2 - Großes wav2vec Modell

Dieses Notebook erkennt mit einem großen Modell von 1GB Größe eine heruntergeladene Testdatei.

Grundsätzlicher Algorithmus von Facebook/Meta: https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/

Leicht einzusetzende Modelle mit Code von Hugging Face: https://huggingface.co/models?pipeline_tag=automatic-speech-recognition&sort=downloads

Mit Laufzeittyp GPU starten, die Zeitunterschiede zu sehen.

---

Allgemein:

Dieses Notebook läuft auf den Servern von Google, es verändert nichts auf Ihrem lokalen Rechner. Alle Eingaben werden Beendigung gelöscht.

Colab Notebooks enthalten Text- oder Codeblöcke. Codeblöcke werden durch SHIFT-ENTER ausgeführt (oder Play-Symbol oben links klicken).

Solange ein Block ausgeführt wird, wird nicht mehr die Nr [3] des Blocks, sondern ein Kreis angezeigt.

Sie können beliebig Daten verändern, das geschieht nur für Sie.

In [ ]:
# Grundsätzliche Libs installieren für Audios und Oberfläche
!pip install torchaudio ipywebrtc
!sudo apt update
!sudo apt install ffmpeg
!jupyter nbextension enable --py widgetsnbextension

# Audio nutzen
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio

# Recording ermöglichen
from IPython.display import Audio
from google.colab import output
output.enable_custom_widget_manager()

# Testdateien laden
!wget http://ainblick.de/test/oop1.wav
!wget http://ainblick.de/test/oop2.wav

In [ ]:
# Simple Lib installieren von: https://github.com/jonatasgrosman/asrecognition
!pip install asrecognition

In [ ]:
from asrecognition import ASREngine
import warnings
warnings.filterwarnings('ignore')
import time

start = time.time()
# Modell von Hugging Face herunterladen
asr = ASREngine("de", model_path="jonatasgrosman/wav2vec2-large-xlsr-53-german")
end = time.time()
print("Dauer: ", end - start)

audio_paths = ["oop2.wav"]
start = time.time()
transcriptions = asr.transcribe(audio_paths)
end = time.time()
print("Dauer: ", end - start)

transcriptions

In [ ]:
# selbst aufnehmen
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -acodec pcm_s16le -ar 16000 recorded.wav -loglevel panic -y
sig, sr = torchaudio.load("recorded.wav")
print(sig.shape)
Audio(data=sig, rate=sr)

In [ ]:
audio_paths = ["recorded.wav"]
transcriptions = asr.transcribe(audio_paths)

transcriptions

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC


model_id = "jonatasgrosman/wav2vec2-large-xlsr-53-german"
processor = Wav2Vec2Processor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id).to("cuda") # .to() für GPU


In [ ]:
start = time.time()
audio, rate = torchaudio.load("recorded.wav")
input_values = processor(audio, return_tensors = "pt", sampling_rate=rate).input_values
logits = None
with torch.no_grad():
    logits = model(input_values[0].to("cuda")).logits # für GPU
    # logits = model(input_values[0]).logits # für CPU
predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)
end = time.time()
print("Dauer: ", end - start)
print("Ergebnis: " + transcription[0])

In [ ]:
print(predicted_ids[0])
print()
i = 1
for s in sorted(processor.tokenizer.get_vocab()):
  print(str(i) + " -> " + str(s))
  i += 1